# Extraçao e Validação dos Dados

## Importando as Bibliotecas

In [53]:
import pandas as pd
import pandera as pa

## Extraindo o arquivo 


In [54]:
dataframe = pd.read_csv('data/ocorrencia_2010_2020.csv',  sep=';', parse_dates=['ocorrencia_dia'], dayfirst=True)
dataframe.head()

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,***,***,RIO DE JANEIRO,RJ,...,2010-01-03,12:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,...,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,...,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,...,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO


### Removendo as colunas que nao iremos usar para a analise

In [55]:
dataframe = dataframe.drop(
    columns=[
        'codigo_ocorrencia1','codigo_ocorrencia3',
        'codigo_ocorrencia4','ocorrencia_latitude',
        'ocorrencia_longitude','investigacao_aeronave_liberada',
        'investigacao_status','divulgacao_relatorio_numero',
        'divulgacao_relatorio_publicado','divulgacao_dia_publicacao',
        'total_aeronaves_envolvidas','ocorrencia_saida_pista'
        ]
    )
dataframe.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,BRASIL,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,BRASIL,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,BRASIL,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,BRASIL,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,BRASIL,SBPK,2010-01-05,19:25:00,0


## Validando os dados

Cria-se um esquema para validar o formato esperado de cada atributo do dataframe.

In [56]:
schema = pa.DataFrameSchema(
    columns={
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        'ocorrencia_classificacao': pa.Column(pa.String),
        'ocorrencia_cidade': pa.Column(pa.String),
        'ocorrencia_uf': pa.Column(pa.String, pa.Check.str_length(2,2)),
        'ocorrencia_pais': pa.Column(pa.String),
        'ocorrencia_aerodromo': pa.Column(pa.String),
        'ocorrencia_dia': pa.Column(pa.DateTime),
        'ocorrencia_hora': pa.Column(pa.String, 
            pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        'total_recomendacoes': pa.Column(pa.Int) 
    }
)

In [57]:
schema.validate(dataframe)

SchemaError: <Schema Column(name=ocorrencia_uf, type=str)> failed element-wise validator 0:
<Check str_length: str_length(2, 2)>
failure cases:
   index failure_case
0   2227          ***

Quando o algum valor do atributo não esta no format esperado pelo schema, ele para a validação, retornando o index do valor que nao esta coforme o formato estabelecido. O dataframe utilizado no projeto, não passou pela primeira validação.
``` 
SchemaError: <Schema Column(name=ocorrencia_uf, type=str)> failed element-wise validator 0:
<Check str_length: str_length(2, 2)>
failure cases:
   index failure_case
0   2227
```


Em seguida, trata-se o erro retornado, para validar novamente o dataframe

In [58]:
dataframe['ocorrencia_uf']= dataframe['ocorrencia_uf'].replace(['***','**'])
dataframe.head()

In [60]:
schema.validate(dataframe)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,BRASIL,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,BRASIL,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,BRASIL,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,BRASIL,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,BRASIL,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,BRASIL,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,BRASIL,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,BRASIL,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,BRASIL,****,2020-12-31,09:00:00,0


In [61]:
dataframe.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_pais                     object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object